In [1]:
import requests
import json
import urllib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
import re
import numpy as np

In [2]:
host = "sofascore.p.rapidapi.com/"
key = "2272a5bfe7mshc5f760247ed1036p1889ccjsnfbfe7887e2ca"

headers = {
    "X-RapidAPI-Key": key,
    "X-RapidAPI-Host": host
}

sofascore_api = "https://" + host
name = "Khvicha Kvaratskhelia"

In [4]:
def get_player_id(name):
    search_url = sofascore_api + "players/search"
    search_params = {"name": name}
    search_response = requests.get(search_url, headers=headers, params=search_params)
    search_response_json = search_response.json()
    player_id = search_response_json['players'][0]['id']
    return player_id

player_id = get_player_id(name)
player_id

<Response [429]>


KeyError: 'players'

In [ ]:
def get_player_data(player_id):
    player_url = sofascore_api + "players/detail"
    player_params = {"playerId": player_id}
    player_response = requests.get(player_url, headers=headers, params=player_params)
    player_response_json = player_response.json()
    return player_response_json

get_player_data(player_id)

In [ ]:
def get_last_games(player_id):
    games_url = sofascore_api + "players/get-last-matches"
    games_params = {"playerId": player_id, 'pageIndex': '0'}
    games_response = requests.get(games_url, headers=headers, params=games_params)
    games_response_json = games_response.json()
    return games_response_json

games_response_json = get_last_games(player_id)
games_response_json

In [ ]:
def get_last_match_ids(games_response_json):
    st = []
    for e in games_response_json['events']:
        st.append(e['id'])
    return st
    
match_id = get_last_match_ids(games_response_json)[-1]
match_id

In [ ]:
def get_game_stats(match_id):
    game_url = sofascore_api + "matches/get-statistics"
    game_params = {"matchId": match_id}
    game_response = requests.get(game_url, headers=headers, params=game_params)
    game_response_json = game_response.json()
    return game_response_json

game_response_json = get_game_stats(match_id)
game_response_json

In [ ]:
s = set()
for stats in game_response_json['statistics'][0]['groups']: #have to know if team that player is on is home or away
    for stat in stats['statisticsItems']:
        s.add(stat['name']) 
s

In [ ]:
def get_player_game_stats(match_id, player_id):
    player_game_url = sofascore_api + 'matches/get-player-statistics'
    player_game_params = {"matchId": match_id, "playerId": player_id}
    player_game_response = requests.get(player_game_url, headers=headers, params=player_game_params)
    player_game_response_json = player_game_response.json()
    return player_game_response_json
get_player_game_stats(match_id, player_id)

In [ ]:
def get_player_game_heatmap(match_id, player_id):
    player_heatmap_url = sofascore_api + 'matches/get-player-heatmap'
    player_heatmap_params = {"matchId": match_id, "playerId": player_id}
    player_heatmap_response = requests.get(player_heatmap_url, headers=headers, params=player_heatmap_params)
    player_heatmap_response_json = player_heatmap_response.json()
    return player_heatmap_response_json
get_player_game_stats(match_id, player_id)

In [ ]:
def get_last_player_games_features(player_id):
    player_games_features = []
    
    last_games = get_last_games(player_id)
    last_games_ids = get_last_match_ids(last_games)
    
    key_stats = set()
    player_game_stats_list = []
    
    for match_id in last_games_ids[::-1]:
        player_game_stats = get_player_game_stats(match_id, player_id)
        player_game_stats_list.append(player_game_stats)
        if player_game_stats.get('statistics'):
            for k in player_game_stats['statistics']:
                key_stats.add(k)
                
    for player_game_stats in player_game_stats_list:
        player_game_key_stats = []
        if player_game_stats.get('statistics'):
            for k in key_stats:
                if k in player_game_stats['statistics']:
                    feature_value = player_game_stats['statistics'][k]
                    if k == 'ratingVersions':
                        player_game_key_stats.append(feature_value['original'])
                    else:
                        player_game_key_stats.append(feature_value) #might be other dict cases
                else:
                    player_game_key_stats.append(0)
                    
        if player_game_key_stats:
            player_games_features.append(player_game_key_stats)
        else:
            player_games_features.append([-1 for _ in range(len(key_stats))])
        
    return player_games_features, key_stats

player_games_features, key_stats = get_last_player_games_features(player_id)
print(key_stats)

In [ ]:
player_games_features[0]

In [ ]:
def get_player_features(player_id): 
    player_features = []
    #position, height, primaryUniqueTournament -> name, proposedMarketValue, preferredFoot
    return player_features

In [ ]:
def get_heatmap_features(player_id):
    heatmap_features = []
    
    last_games = get_last_games(player_id)
    last_games_ids = get_last_match_ids(last_games)
    
    min_x = min_y = np.inf
    max_x = max_y = 0
    
    for match_id in last_games_ids[::-1]:
        heatmap_stats = get_player_game_heatmap(match_id, player_id)
        
        heatmap_key_stats = []
        if heatmap_stats.get('heatmap'):
            x_coords = []
            y_coords = []
            for coord in heatmap_stats['heatmap']:
                x_coords.append(coord['x'])
                y_coords.append(coord['y'])
                
                max_x = max(x, max_x)
                max_y = max(y, max_y)
                
                min_x = min(x, min_x)
                min_y = min(y, min_y)
                
            heatmap_key_stats += [np.mean(x_coords), np.mean(y_coords), np.var(x_coords), np.var(y_coords)]
        
        if heatmap_key_stats:
            heatmap_features.append(heatmap_key_stats)
        else:
            heatmap_features.append([-1, -1, -1, -1])
            
    print(min_x, min_y, max_x, max_y)
    return heatmap_features

heatmap_features = get_heatmap_features(player_id)

In [ ]:
heatmap_features[0]

In [ ]:
def mean_column_vals(arr, j):
    vals = []
    for row in arr:
        if j < len(row):
            vals.append(row[j])
    return np.mean(vals)

In [ ]:
def fill_in_empty_vals(arr):
    for j in range(len(arr[0])):
        mean_val = mean_column_vals(arr, j)
        for i in range(len(arr)):
            if arr[i][j] == -1:
                arr[i][j] = mean_val
    return arr

In [ ]:
nonempty_player_games_features = fill_in_empty_vals(player_games_features)
nonempty_heatmap_features = fill_in_empty_vals(heatmap_features)